Importamos las librerías

In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

Cargamos el dataset de train

In [3]:
train_set = pd.read_csv("train_set.csv", index_col=0)

In [4]:
train_set

,id_img,paths,label
0,100,data\train\happy\100.jpg,happy
1,10000,data\train\happy\10000.jpg,happy
2,10001,data\train\happy\10001.jpg,happy
3,10005,data\train\happy\10005.jpg,happy
4,10035,data\train\happy\10035.jpg,happy
...,...,...,...
12097,9966,data\train\sadness\9966.jpg,sadness
12098,9974,data\train\sadness\9974.jpg,sadness
12099,9976,data\train\sadness\9976.jpg,sadness
12100,9986,data\train\sadness\9986.jpg,sadness


Cargamos el dataset de test

In [5]:
train_set['arrays'] = train_set['paths'].apply(lambda x: cv2.imread(x))

In [6]:
train_set

,id_img,paths,label,arrays
0,100,data\train\happy\100.jpg,happy,"[[[162, 162, 162], [163, 163, 163], [158, 158,..."
1,10000,data\train\happy\10000.jpg,happy,"[[[134, 134, 134], [127, 127, 127], [134, 134,..."
2,10001,data\train\happy\10001.jpg,happy,"[[[253, 253, 253], [253, 253, 253], [247, 247,..."
3,10005,data\train\happy\10005.jpg,happy,"[[[183, 183, 183], [184, 184, 184], [183, 183,..."
4,10035,data\train\happy\10035.jpg,happy,"[[[249, 249, 249], [235, 235, 235], [46, 46, 4..."
...,...,...,...,...
12097,9966,data\train\sadness\9966.jpg,sadness,"[[[97, 97, 97], [137, 137, 137], [161, 161, 16..."
12098,9974,data\train\sadness\9974.jpg,sadness,"[[[60, 60, 60], [25, 25, 25], [27, 27, 27], [2..."
12099,9976,data\train\sadness\9976.jpg,sadness,"[[[44, 44, 44], [44, 44, 44], [45, 45, 45], [4..."
12100,9986,data\train\sadness\9986.jpg,sadness,"[[[109, 109, 109], [23, 23, 23], [21, 21, 21],..."


In [9]:
test_set = pd.read_csv("test_set.csv", index_col=0)
test_set['arrays'] = test_set['paths'].apply(lambda x: cv2.imread(x))
test_set

,id_img,paths,arrays
0,10019,data/test/10019.jpg,"[[[159, 159, 159], [157, 157, 157], [158, 158,..."
1,10023,data/test/10023.jpg,"[[[195, 195, 195], [198, 198, 198], [193, 193,..."
2,10074,data/test/10074.jpg,"[[[105, 105, 105], [40, 40, 40], [45, 45, 45],..."
3,10096,data/test/10096.jpg,"[[[255, 255, 255], [253, 253, 253], [253, 253,..."
4,10106,data/test/10106.jpg,"[[[6, 6, 6], [4, 4, 4], [5, 5, 5], [7, 7, 7], ..."
...,...,...,...
2959,9864,data/test/9864.jpg,"[[[13, 13, 13], [12, 12, 12], [7, 7, 7], [7, 7..."
2960,987,data/test/987.jpg,"[[[194, 194, 194], [199, 199, 199], [200, 200,..."
2961,9885,data/test/9885.jpg,"[[[29, 29, 29], [23, 23, 23], [22, 22, 22], [1..."
2962,9889,data/test/9889.jpg,"[[[246, 246, 246], [245, 245, 245], [247, 247,..."


Creamos nuestra X, tendremos que quedarnos unicamente con un color por pixel.

Para ello utilizamos el método stack() para aplanar las imágenes, es decir, convertir los pixeles en un array 2D.

In [10]:
from sklearn import preprocessing

train_set[train_set.label == "happy"].index
train_set[train_set.label == "sadness"].index

# aplico label enconder para convertir en numeros happy y sadness
label_enconder = preprocessing.LabelEncoder()
label_enconder.fit(train_set.label)
y = label_encoder.transform(train_set.label)

y = y.reshape(-1,1) # la transformo en matriz de 1D
y.shape

X = train_set['arrays']
X = np.array(X)
X = np.stack(X)

X_test = test_set['arrays']
X_test = np.array(X_test)
X_test = np.stack(X_test) # une todas las arrays de cada fila

Escalamos la X para que los datos estén entre 0 y 1.

In [11]:
X_scaled = X/255 
X_test_scaled = X_test/255

Dividimos entre train y test

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [13]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9681, 48, 48, 3)
(2421, 48, 48, 3)
(9681, 1)
(2421, 1)


Como los modelos de regresión me daban error y en el código de Marta, he visto tensorflow, he investigado y he trabajado el modelo usándolo

In [14]:
from tensorflow import keras
vgg = keras.applications.VGG16(weights='imagenet', include_top=False)
vgg.input_shape

58889256/58889256 [==============================] - 11s 0us/step


(None, None, None, 3)

In [15]:
model = keras.models.Sequential()

model.add(keras.layers.GaussianNoise(.2,input_shape=(48, 48,3)))
model.add(vgg) # convolution layers


model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(.4)) # para evitar overfitting
model.add(keras.layers.Dense(units = 512,
                            activation='relu'))

model.add(keras.layers.Dropout(.4))  # para evitar overfitting
model.add(keras.layers.BatchNormalization())# para acelerar el proceso
model.add(keras.layers.Dense(units = 256,
                            activation='relu'))

model.add(keras.layers.Dropout(.2))  # para evitar overfitting
model.add(keras.layers.Dense(units = 2,
                            activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gaussian_noise (GaussianNoi  (None, 48, 48, 3)        0         
 se)                                                             
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                        

In [16]:
model.compile(
    optimizer = keras.optimizers.Adam(1e-5),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = [keras.metrics.SparseCategoricalAccuracy()]
)

In [17]:
earlystop = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5, verbose=1,
    mode='auto', restore_best_weights=True
)

In [18]:
history = model.fit(
    X_train,
    y_train,
    batch_size = 64,
    epochs = 60,
    validation_data = (X_test,y_test),
    callbacks=[earlystop] #para evitar overfitting
)

Epoch 1/60
152/152 [==============================] - 427s 3s/step - loss: 0.8625 - sparse_categorical_accuracy: 0.5267 - val_loss: 0.6180 - val_sparse_categorical_accuracy: 0.6605
Epoch 2/60
152/152 [==============================] - 433s 3s/step - loss: 0.7779 - sparse_categorical_accuracy: 0.5901 - val_loss: 0.5636 - val_sparse_categorical_accuracy: 0.7080
Epoch 3/60
152/152 [==============================] - 423s 3s/step - loss: 0.7119 - sparse_categorical_accuracy: 0.6389 - val_loss: 0.5429 - val_sparse_categorical_accuracy: 0.7323
Epoch 4/60
152/152 [==============================] - 415s 3s/step - loss: 0.6065 - sparse_categorical_accuracy: 0.7062 - val_loss: 0.6181 - val_sparse_categorical_accuracy: 0.6708
Epoch 5/60
152/152 [==============================] - 431s 3s/step - loss: 0.5492 - sparse_categorical_accuracy: 0.7381 - val_loss: 0.4180 - val_sparse_categorical_accuracy: 0.8150
Epoch 6/60
152/152 [==============================] - 439s 3s/step - loss: 0.4922 - sparse_cate

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("final accuracy : ",test_acc)

76/76 [==============================] - 20s 259ms/step - loss: 0.2867 - sparse_categorical_accuracy: 0.8798
final accuracy :  0.8798017501831055


In [21]:
predictions = model.predict(X_test_scaled).round(3)

93/93 [==============================] - 25s 262ms/step


In [23]:
predictions.argmax(axis=1)
le.inverse_transform(predictions.argmax(axis=1))
submission_prediction = pd.DataFrame(data = test_set, columns = ['id_img'])
submission_prediction['label'] = le.inverse_transform(predictions.argmax(axis=1))
submission_prediction.to_csv('submission.csv', index=False)